# INSE 6220 — Full Error-Free Colab Notebook
This notebook includes all required imports, installations, preprocessing, PCA, PyCaret modeling, and plotting.

In [ ]:
!pip install seaborn scikit-learn pycaret shap matplotlib pandas numpy --quiet
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pycaret.classification import *
import shap
print('All libraries loaded successfully!')

In [ ]:
# Load your dataset (Upload manually or replace with path)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])
print(df.head())

In [ ]:
# Preprocessing
target = 'target'
X = df.drop(columns=[target])
y = df[target]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print('Scaling complete.')

In [ ]:
# PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)
explained = pca.explained_variance_ratio_
cum = np.cumsum(explained)
plt.plot(explained, 'o-', label='Variance')
plt.plot(cum, 's--', label='Cumulative')
plt.legend(); plt.title('Scree Plot'); plt.show()

In [ ]:
# PyCaret Setup
clf = setup(df, target=target, session_id=123, silent=True, html=False)
best_model = compare_models()
print(best_model)

In [ ]:
# Evaluate best model
eval_results = predict_model(best_model)
cm = confusion_matrix(eval_results[target], eval_results['Label'])
disp = ConfusionMatrixDisplay(cm)
disp.plot(cmap='viridis'); plt.show()

In [ ]:
# SHAP Explainability
explainer = shap.Explainer(best_model)
shap_values = explainer(X)
shap.summary_plot(shap_values, X)